In [1]:
import pandas as pd
import csv
import json
import numpy as np
monthID = "0623"
startMonthID = "0423"
endMonthID = "0623"

In [2]:
with open(f'C:/Users/weijin.ang/Code/Transactions/resi_transactions_{startMonthID}_{endMonthID}.json', 'r') as f:
  data = json.load(f)
#data

In [3]:
# normalize events
df = pd.json_normalize(data, 'Result')

# explode all columns with lists of dicts
df = df.apply(lambda x: x.explode()).reset_index(drop=True)  # df.apply(pd.Series.explode).reset_index(drop=True) also works

# list of columns with dicts
cols_to_normalize = ['rental']

# if there are keys, which will become column names, overlap with excising column names
# add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

# combine df with the normalized columns , concat both lists
df = pd.concat([df] + normalized, axis=1).drop(columns=cols_to_normalize)

In [4]:
#df
df.to_csv(f'C:/Users/weijin.ang/Code/Transactions/transaction_resi_converted_raw_csv_{startMonthID}_{endMonthID}.csv', index=False)
df = df[df.rental_propertyType.isin(["Non-landed Properties", "Executive Condominium"])]
df = df[df.rental_noOfBedRoom.isin(["1","2","3","4","5"])]
df = df[df.rental_leaseDate.isin([monthID])]
df['rental_areaSqft'] = np.where(df['rental_areaSqft'] == '>3000', '3000-3000', df['rental_areaSqft'])
df[['rental_areaSqft_lower', 'rental_areaSqft_higher']] = df['rental_areaSqft'].str.split('-', expand=True)

#df = pd.concat([df, df['rental_areaSqft'].str.split('-', expand=True)], axis=1)
df = df.astype({"rental_areaSqft_lower":"int","rental_areaSqft_higher":"int"})
df.dtypes
#df
col = df.loc[: , "rental_areaSqft_lower":"rental_areaSqft_higher"]
df['mean_areaSqft'] = col.mean(axis=1)
df

,street,x,project,y,rental_areaSqm,rental_leaseDate,rental_propertyType,rental_district,rental_areaSqft,rental_noOfBedRoom,rental_rent,rental_areaSqft_lower,rental_areaSqft_higher,mean_areaSqft
0,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,6500,2300,2400,2350.0
2,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,120-130,0623,Non-landed Properties,15,1300-1400,3,5200,1300,1400,1350.0
4,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,8050,2300,2400,2350.0
9,HILLVIEW AVENUE,19806.57039,MERAWOODS,37591.36252,90-100,0623,Non-landed Properties,23,1000-1100,2,3300,1000,1100,1050.0
10,HILLVIEW AVENUE,19806.57039,MERAWOODS,37591.36252,120-130,0623,Non-landed Properties,23,1300-1400,3,4200,1300,1400,1350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20173,PUNGGOL FIELD,35969.63449,PRIVE,42560.406,70-80,0623,Executive Condominium,19,700-800,2,4300,700,800,750.0
20174,PUNGGOL FIELD,35969.63449,PRIVE,42560.406,70-80,0623,Executive Condominium,19,800-900,2,3850,800,900,850.0
20179,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0623,Non-landed Properties,08,700-800,3,5000,700,800,750.0
20183,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0623,Non-landed Properties,08,700-800,3,5100,700,800,750.0


In [5]:
df = df.groupby(['project', 'rental_noOfBedRoom']).mean().reset_index()
df
df = df.astype({"mean_areaSqft":"str"})
df.dtypes
df['mean_areaSqft'] = np.where(df['mean_areaSqft'] == '3000.0', '>3000', df['mean_areaSqft'])

C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_6772\3294160895.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['project', 'rental_noOfBedRoom']).mean().reset_index()


In [6]:
df = df.rename(columns={"rental_noOfBedRoom": "noOfBedRoom"})
df = df.drop(['rental_areaSqft_lower', 'rental_areaSqft_higher'], axis=1)

In [7]:
df.to_csv(f'C:/Users/weijin.ang/Code/Transactions/transactions_resi_project_bedroom_index_{monthID}.csv', na_rep='N/A', quoting=csv.QUOTE_NONE, index=False)